In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path
from functools import reduce

In [3]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: elmiraarifullina
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews


100%|██████████| 15.1M/15.1M [00:00<00:00, 106MB/s] 

In [4]:
import json


n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774865


In [5]:
def mapper(path):
    # Ваш код
    try:
      with open(path, 'r') as f:
        info = json.load(f)
        score = float(info['movieIMDbRating'])
        return (1, score, 0)
    except (IsADirectoryError, json.JSONDecodeError, KeyError):
      return (0, 0.0, 0)
    return (score, )


def reducer(score_data1, score_data2):
    #  Ваш код
    n1, mean1, M2_1 = score_data1
    n2, mean2, M2_2 = score_data2

    n = n1 + n2
    if n == 0:
      return (0, 0.0, 0)

    delta = mean2 - mean1
    mean = mean1 + delta * n2 / n
    M2 = M2_1 + M2_2 +delta ** 2 * n1 * n2 / n
    return n, mean, M2

In [6]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774865
CPU times: user 168 ms, sys: 167 ms, total: 335 ms
Wall time: 339 ms
